In [1]:
%pylab inline
import GPy
import GPyOpt
import numpy as np
import matplotlib.pyplot as plt
from GPyOpt.util.pareto import ParetoFront
from GPyOpt.experiment_design import initial_design
from GPyOpt.acquisitions.HvEI import AcquisitionHvEI
from GPyOpt.methods import ModularBayesianMultiobjectiveOptimization

Populating the interactive namespace from numpy and matplotlib


In [2]:
def binhKorn(x,y):
    f1 = 4.*x*x + 4*y*y
    f2 = (x-5.)**2 + (y-5.)**2
    return np.array([f1,f2])

def disc_constraint(x,y):
    return (25.-(x-5)**2-y**2)

def line_constraint(x,y):
    return -(x-3*y+10)

In [3]:
np.set_printoptions(precision=6)

objective = None 

space =[{'name': 'var_1', 'type': 'continuous', 'domain': (-0.0,5.0)},
        {'name': 'var_2', 'type': 'continuous', 'domain': (-0.0,3.0)}]
space = GPyOpt.Design_space(space = space)

kernel   = None 
kernel_c = None

model_f1 = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False)
model_f2 = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False)
model_f  = [model_f1,model_f2]

model_c1 = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False)
model_c2 = GPyOpt.models.GPModel(optimize_restarts=5,verbose=False)
model_c  = [model_c1,model_c2]

aquisition_optimizer = GPyOpt.optimization.AcquisitionOptimizer(space)

num_init = 23
typ_init = 'latin'
X_init = initial_design(typ_init,space,num_init)

Y_init = np.zeros((X_init.shape[0],2))
C_init = np.zeros((X_init.shape[0],2))

for i in range(num_init):
    Y_init[i,0],Y_init[i,1] = binhKorn(X_init[i,0],X_init[i,1])
    C_init[i,0] = disc_constraint(X_init[i,0],X_init[i,1])
    C_init[i,1] = line_constraint(X_init[i,0],X_init[i,1])

np.set_printoptions(precision=4)
print(np.hstack((X_init,Y_init,C_init)))

[[ 2.0652e+00  1.1087e+00  2.1977e+01  2.3755e+01  1.5158e+01 -8.7391e+00]
 [ 1.4130e+00  2.5435e+00  3.3864e+01  1.8901e+01  5.6645e+00 -3.7826e+00]
 [ 1.0870e-01  1.8913e+00  1.4355e+01  3.3589e+01 -2.5019e+00 -4.4348e+00]
 [ 2.2826e+00  2.2826e+00  4.1682e+01  1.4768e+01  1.2405e+01 -5.4348e+00]
 [ 4.8913e+00  3.2609e-01  9.6125e+01  2.1857e+01  2.4882e+01 -1.3913e+01]
 [ 4.4565e+00  4.5652e-01  8.0276e+01  2.0939e+01  2.4496e+01 -1.3087e+01]
 [ 4.2391e+00  7.1739e-01  7.3940e+01  1.8920e+01  2.3906e+01 -1.2087e+01]
 [ 3.3696e+00  2.6739e+00  7.4015e+01  8.0690e+00  1.5192e+01 -5.3478e+00]
 [ 2.5000e+00  2.1522e+00  4.3527e+01  1.4360e+01  1.4118e+01 -6.0435e+00]
 [ 9.7826e-01  2.0217e+00  2.0178e+01  2.5044e+01  4.7382e+00 -4.9130e+00]
 [ 3.5870e+00  5.8696e-01  5.2843e+01  2.1472e+01  2.2659e+01 -1.1826e+01]
 [ 3.8043e+00  1.6304e+00  6.8526e+01  1.2784e+01  2.0912e+01 -8.9130e+00]
 [ 2.7174e+00  1.2391e+00  3.5679e+01  1.9354e+01  1.8254e+01 -9.0000e+00]
 [ 1.6304e+00  1.5000e+00

In [4]:
iter_count = 150-num_init
current_iter = 0
X_step = X_init
Y_step = Y_init
C_step = C_init 

while current_iter < iter_count:
    Pareto = ParetoFront(Y_step)
    
    print(Pareto)
    
    acquisition = AcquisitionHvEI(model_f,space,optimizer=aquisition_optimizer, jitter = 1e-3,
                                  model_c=model_c,jitter_c = np.array([0.0,0.0]),P=Pareto,r=np.array([140,60]))
    
    # --- CHOOSE a collection method
    evaluator = GPyOpt.core.evaluators.Sequential(acquisition)
    
    bo_step = ModularBayesianMultiobjectiveOptimization(model_f, space, objective, acquisition, evaluator, 
                                                        X_init = X_step, Y_init = Y_step, C_init = C_step, 
                                                        model_c = model_c, normalize_Y = False)
    
    x_next = bo_step.suggest_next_locations()
    y_next = binhKorn(x_next[0][0],x_next[0][1])
    c1_next = disc_constraint(x_next[0][0],x_next[0][1])
    c2_next = line_constraint(x_next[0][0],x_next[0][1])
    c_next = np.array([c1_next,c2_next])
    print("Current iteration - ",current_iter," : ",np.array([x_next[0][0],x_next[0][1],y_next,c_next[0],c_next[1]]))
    
    X_step = np.vstack((X_step, x_next))
    Y_step = np.vstack((Y_step, y_next))
    C_step = np.vstack((C_step, c_next))
    
    current_iter += 1

[[  2.4688  41.052 ]
 [  5.0095  36.035 ]
 [  8.5936  31.7136]
 [ 19.6333  23.604 ]
 [ 26.0605  20.4282]
 [ 33.8639  18.9008]
 [ 41.6824  14.7684]
 [ 43.5274  14.3601]
 [ 68.5255  12.7836]
 [ 74.0151   8.069 ]
 [ 87.9887   7.6493]
 [121.8336   4.3715]]
(1000, 2)
[[20.5598]
 [19.4252]
 [ 6.6122]
 [ 5.4765]
 [ 2.3294]
 [ 9.3584]
 [ 6.4977]
 [ 2.0489]
 [11.411 ]
 [ 5.5133]
 [ 2.3204]
 [18.6117]
 [16.026 ]
 [17.1947]
 [ 8.3943]
 [29.6127]
 [ 5.3762]
 [64.8672]
 [ 7.0186]
 [ 5.0754]
 [ 5.5529]
 [ 3.7696]
 [ 7.0673]
 [22.7252]
 [ 4.9145]
 [ 7.5817]
 [34.3032]
 [12.4847]
 [ 2.5922]
 [ 5.3944]
 [ 5.9004]
 [49.4295]
 [29.1152]
 [16.3056]
 [12.6642]
 [24.3973]
 [12.8505]
 [ 3.0532]
 [ 5.8738]
 [ 5.2415]
 [63.0826]
 [20.3902]
 [ 7.1518]
 [ 5.401 ]
 [12.4949]
 [12.9958]
 [28.0252]
 [ 1.4648]
 [15.2964]
 [ 1.4271]
 [19.2925]
 [23.0637]
 [ 5.891 ]
 [ 7.7414]
 [10.4572]
 [21.7242]
 [58.9043]
 [15.4086]
 [52.5137]
 [ 9.2719]
 [ 5.1739]
 [ 6.7888]
 [ 4.6791]
 [14.0098]
 [ 5.4351]
 [ 1.75  ]
 [ 2.3917]


[[2.575 ]
 [2.4772]
 [1.0949]
 [1.0082]
 [0.502 ]
 [1.4478]
 [1.0747]
 [0.4498]
 [1.7385]
 [0.9857]
 [0.5003]
 [2.3043]
 [2.1751]
 [2.2225]
 [1.3728]
 [3.1902]
 [0.9677]
 [4.6368]
 [1.207 ]
 [0.9264]
 [1.019 ]
 [0.7485]
 [1.2257]
 [2.7353]
 [0.925 ]
 [1.273 ]
 [3.428 ]
 [1.829 ]
 [0.5454]
 [0.9963]
 [1.015 ]
 [4.1611]
 [3.1697]
 [2.216 ]
 [1.8463]
 [2.8447]
 [1.8573]
 [0.6298]
 [1.0387]
 [0.9742]
 [4.5934]
 [2.5649]
 [1.2368]
 [0.9872]
 [1.8066]
 [1.8837]
 [2.98  ]
 [0.3344]
 [2.0918]
 [0.3263]
 [2.4235]
 [2.7015]
 [1.01  ]
 [1.3111]
 [1.6347]
 [2.6261]
 [4.4757]
 [2.1339]
 [4.2684]
 [1.447 ]
 [0.9544]
 [1.1655]
 [0.8901]
 [1.949 ]
 [0.9987]
 [0.391 ]
 [0.5129]
 [0.7169]
 [1.2689]
 [3.6009]
 [0.8356]
 [2.2018]
 [0.6001]
 [4.0973]
 [1.4988]
 [1.0972]
 [4.1236]
 [1.5148]
 [3.464 ]
 [2.651 ]
 [0.6983]
 [1.8683]
 [1.0404]
 [0.5357]
 [4.4398]
 [0.3664]
 [4.3287]
 [2.2961]
 [0.6008]
 [0.6742]
 [1.5008]
 [2.2729]
 [2.1651]
 [1.2377]
 [0.7969]
 [2.3767]
 [1.1845]
 [0.8131]
 [2.876 ]
 [0.9903]


[[1.3996]
 [1.3639]
 [0.7429]
 [0.7237]
 [0.4143]
 [0.9254]
 [0.7302]
 [0.378 ]
 [1.0791]
 [0.6996]
 [0.413 ]
 [1.2702]
 [1.2506]
 [1.251 ]
 [0.9038]
 [1.5843]
 [0.6902]
 [1.859 ]
 [0.8241]
 [0.6679]
 [0.7295]
 [0.5731]
 [0.8386]
 [1.451 ]
 [0.6766]
 [0.855 ]
 [1.6379]
 [1.1101]
 [0.4416]
 [0.717 ]
 [0.7058]
 [1.8032]
 [1.5814]
 [1.271 ]
 [1.117 ]
 [1.4822]
 [1.119 ]
 [0.4989]
 [0.7297]
 [0.7048]
 [1.8566]
 [1.3971]
 [0.8443]
 [0.7077]
 [1.0928]
 [1.134 ]
 [1.4937]
 [0.293 ]
 [1.2131]
 [0.2867]
 [1.3321]
 [1.423 ]
 [0.7016]
 [0.881 ]
 [1.0336]
 [1.4051]
 [1.8453]
 [1.2397]
 [1.8185]
 [0.9283]
 [0.6894]
 [0.7982]
 [0.6569]
 [1.1493]
 [0.7169]
 [0.335 ]
 [0.4214]
 [0.5543]
 [0.8591]
 [1.6869]
 [0.6171]
 [1.2549]
 [0.4799]
 [1.7926]
 [0.971 ]
 [0.7703]
 [1.7961]
 [0.9765]
 [1.6573]
 [1.398 ]
 [0.5404]
 [1.1312]
 [0.7313]
 [0.4374]
 [1.8403]
 [0.3173]
 [1.8271]
 [1.2869]
 [0.4796]
 [0.5273]
 [0.9711]
 [1.2867]
 [1.2522]
 [0.8391]
 [0.5993]
 [1.3232]
 [0.8038]
 [0.6124]
 [1.471 ]
 [0.7093]


[[0.0983]
 [0.097 ]
 [0.0649]
 [0.0656]
 [0.0428]
 [0.0761]
 [0.0639]
 [0.0398]
 [0.0856]
 [0.0632]
 [0.0427]
 [0.0914]
 [0.0926]
 [0.0915]
 [0.0759]
 [0.1026]
 [0.0626]
 [0.099 ]
 [0.0715]
 [0.0612]
 [0.066 ]
 [0.0552]
 [0.0727]
 [0.0997]
 [0.0624]
 [0.0731]
 [0.1026]
 [0.0865]
 [0.0449]
 [0.0651]
 [0.0629]
 [0.103 ]
 [0.1027]
 [0.0938]
 [0.0868]
 [0.1003]
 [0.0867]
 [0.0496]
 [0.0652]
 [0.0644]
 [0.0995]
 [0.0983]
 [0.073 ]
 [0.0642]
 [0.0851]
 [0.0876]
 [0.0984]
 [0.0321]
 [0.0908]
 [0.0315]
 [0.095 ]
 [0.0977]
 [0.0625]
 [0.0751]
 [0.0833]
 [0.0976]
 [0.1007]
 [0.0925]
 [0.1023]
 [0.0765]
 [0.063 ]
 [0.0696]
 [0.0611]
 [0.0876]
 [0.065 ]
 [0.0359]
 [0.0434]
 [0.0538]
 [0.0738]
 [0.1036]
 [0.0577]
 [0.0924]
 [0.0481]
 [0.1033]
 [0.0801]
 [0.0684]
 [0.1031]
 [0.0802]
 [0.1036]
 [0.0964]
 [0.0526]
 [0.0878]
 [0.0654]
 [0.0447]
 [0.101 ]
 [0.0344]
 [0.1019]
 [0.0934]
 [0.0481]
 [0.0518]
 [0.08  ]
 [0.094 ]
 [0.0931]
 [0.0723]
 [0.0569]
 [0.0952]
 [0.0697]
 [0.0581]
 [0.0985]
 [0.0643]


[[0.0161]
 [0.0161]
 [0.014 ]
 [0.0145]
 [0.0112]
 [0.0151]
 [0.0138]
 [0.0106]
 [0.0161]
 [0.014 ]
 [0.0112]
 [0.0156]
 [0.0161]
 [0.0158]
 [0.0153]
 [0.0153]
 [0.0139]
 [0.012 ]
 [0.015 ]
 [0.0138]
 [0.0145]
 [0.0131]
 [0.0152]
 [0.0159]
 [0.0141]
 [0.0151]
 [0.0149]
 [0.016 ]
 [0.0115]
 [0.0144]
 [0.0138]
 [0.0134]
 [0.0154]
 [0.0162]
 [0.016 ]
 [0.0158]
 [0.0159]
 [0.0123]
 [0.0143]
 [0.0143]
 [0.0122]
 [0.0161]
 [0.0152]
 [0.0142]
 [0.0158]
 [0.016 ]
 [0.0152]
 [0.0091]
 [0.016 ]
 [0.009 ]
 [0.0159]
 [0.0157]
 [0.0137]
 [0.0154]
 [0.016 ]
 [0.0159]
 [0.0125]
 [0.0162]
 [0.0131]
 [0.0152]
 [0.0141]
 [0.0147]
 [0.0139]
 [0.0158]
 [0.0144]
 [0.0099]
 [0.0113]
 [0.013 ]
 [0.0153]
 [0.0146]
 [0.0134]
 [0.016 ]
 [0.0121]
 [0.0136]
 [0.0158]
 [0.0148]
 [0.0135]
 [0.0157]
 [0.0149]
 [0.0156]
 [0.0127]
 [0.0161]
 [0.0143]
 [0.0115]
 [0.0126]
 [0.0096]
 [0.013 ]
 [0.016 ]
 [0.0121]
 [0.0126]
 [0.0157]
 [0.0161]
 [0.0162]
 [0.0151]
 [0.0133]
 [0.0161]
 [0.0147]
 [0.0135]
 [0.0154]
 [0.0143]


[[2.6322]]
[[2.6341]]
[[1.9254]]
[[0.3497]]
[[2.4233]]
[[0.0413]]
[[7.0657]]
[[0.4108]]
[[0.5507]]
[[1.4077]]
[[0.5457]]
[[0.4514]]
[[0.2309]]
[[0.0271]]
[[0.1667]]
[[1.8011]]
[[0.0983]]
[[0.1297]]
[[0.3153]]
[[0.1084]]
[[0.0777]]
[[0.0255]]
[[6.744e-05]]
[[17.2131]]
[[0.8424]]
[[1.0834]]
[[2.4442]]
[[0.6961]]
[[0.4348]]
[[0.1258]]
[[5.5065]]
[[0.2238]]
[[0.2735]]
[[0.5209]]
[[0.0733]]
[[0.0161]]
[[9.545]]
[[0.3343]]
[[0.3921]]
[[0.6576]]
[[0.0395]]
[[12.7174]]
[[0.2983]]
[[0.3127]]
[[0.4413]]
[[2.0573]]
[[0.0125]]
[[0.0047]]
[[0.7839]]
[[0.0012]]
[[0.2084]]
(1, 13, 13)
(1, 13)
(1,)
(1, 1)


UnboundLocalError: local variable 'df_acqu' referenced before assignment